In [10]:
import torch
from torch.utils import data
import torchvision
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [11]:
# import the dataset and loader from data_utils.py
import data_utils

image_folder_path = "data/image"
desired_image_size = (64, 64)

batch_size = 1
# create a dataset so that dataset[i] returns the ith image
rl_data = data_utils.EmojiDataset(image_folder_path, desired_image_size)
# make a dataloader that returns the images as batches for parallel processing
rl_loader = data.DataLoader(rl_data, batch_size)

/home/sanspapyrus683/.local/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [12]:
import models


# functions that save and load the model and optimizer
save_to = "./checkpoints/model.pt"
def save(path, gen, disc, op_g, op_d):
    torch.save(
        {
            "generator_weights" : gen.state_dict(),
            "discriminator_weights" : disc.state_dict(),
            "generator_optimizer_weights" : op_g.state_dict(),
            "discriminator_optimizer_weights" : op_d.state_dict(),
        },
        path
    )


def load(path):
    # initialize 
    dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    checkpoint = torch.load(path)
    gen = models.Generator().to(dev)
    disc = models.Discriminator().to(dev)

    op_d = torch.optim.Adam(gen.parameters(), lr=lr)
    op_g = torch.optim.Adam(disc.parameters(), lr=lr)

    gen.load_state_dict(checkpoint["generator_weights"])
    disc.load_state_dict(checkpoint["discriminator_weights"])
    op_g.load_state_dict(checkpoint["generator_optimizer_weights"])
    op_d.load_state_dict(checkpoint["discriminator_optimizer_weights"])

    return gen, disc, op_g, op_d

In [13]:
generator = models.Generator()
discriminator = models.Discriminator()

# use the gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = generator.to(device)
discriminator = discriminator.to(device)

In [14]:
# Initialize the loss function
criterion = torch.nn.BCELoss()

# Create batch of latent vectors that we will use to visualize the progression of the generator
fixed_noise = torch.randn(64, 100, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# set a learning rate
lr = 0.1

# Setup optimizers for both generator and discriminator
optim_d = torch.optim.AdamW(generator.parameters(), lr=lr)
optim_g = torch.optim.AdamW(discriminator.parameters(), lr=lr)

In [15]:
generator.train()
discriminator.train()

num_epochs = 1
for epoch in tqdm(range(1, num_epochs + 1)):
    for i, data in enumerate(rl_loader):
        ########################################################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        #######################################################
        ## Train with all-real batch
        # Format batch
        data = data.to(device)

        # Forward pass real batch through D
        optim_d.zero_grad()
        logits = discriminator(data)

        # Calculate loss on all-real batch
        loss = criterion(logits, torch.tensor(real_label))

        # Calculate gradients for D in backward pass
        loss.backward()
        optim_d.step()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        fake_amt = 1
        latent_vec = torch.randn(fake_amt, 64, 100, 4, device=device)

        # Generate fake image batch with G
        optim_g.zero_grad()
        fake_img = generator(latent_vec)

        # Classify all fake batch with D
        optim_g.zero_grad()
        logits = discriminator(fake_img)

        # Calculate D's loss on the all-fake batch
        loss = criterion(logits, torch.tensor(fake_label))

        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        loss.backward()

        # Compute error of D as sum over the fake and the real batches

        # Update D

        ########################################################
        # (2) Update G network: maximize log(D(G(z)))
        #######################################################
        
        # Since we just updated D, perform another forward pass of all-fake batch through D

        # Calculate G's loss based on this output

        # Calculate gradients for G

        # Update G

        # # Output training stats
       
        # Save Losses for plotting later

        # Check how the generator is doing by saving G's output on fixed_noise

  0%|          | 0/1 [00:00<?, ?it/s]

AttributeError: 'Discriminator' object has no attribute 'conv'

In [ ]:
# generate images from the model